<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_Create_shareable_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
import requests
import re
from unicodedata import normalize
import os
from zipfile import ZipFile
from google.colab import files
from IPython.display import HTML, display, clear_output

In [ ]:
#@title Enter GetAccept login information
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
BASEURL = "https://api.getaccept.com/v1" #@param {type:"string"}
ENTITY = "" #@param {type: "string"}
#@markdown If no entity id is entered the first/default will be used

In [ ]:
# Make auth request and store auth header
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": ENTITY }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))
else:
  raise Exception("Please provide email and password to the GetAccept entity you want to export documents from")

In [ ]:
# Get template data
url = BASEURL + "/templates"
params = { "showall": "true", "limit": 1, "offset": 0 }
response = requests.get(url, headers=source_auth_headers, params=params)
templates = response.json()["templates"]
template_id = templates[0]["id"]
print("Using template: " + templates[0]["name"])

In [ ]:
# Create new document
payload = {
    "name": "My document",
    "template_id": template_id,
    "recipients": [{
        "email": EMAIL
    }]
}
response = requests.post(BASEURL+"/documents", json=payload, headers=auth_headers)
new_document_data = response.json()
if "id" in new_document_data:
  print("Document created with id: "+new_document_data["id"])
else:
  raise Exception("Could not create document, please check the input data")

In [ ]:
# Seal document
payload = {}
response = requests.post(BASEURL+"/documents/"+new_document_data["id"]+"/seal", json=payload, headers=auth_headers)
seal_data = response.json()
if "message" in seal_data:
  print(seal_data["message"])
else:
  if len(seal_data["recipients"]) > 0:
    print("Document has been sealed!\r\n")
    print("Recipient links:\r\n-", '\r\n- '.join(map(lambda x: x["first_name"]+" "+x["last_name"]+": \t"+x["document_url"], seal_data["recipients"])))
  else:
    raise Exception("Could not seal the document, please check the input data")

In [ ]:
# Additional function to get recipient links
response = requests.get(BASEURL+"/documents/"+new_document_data["id"]+"/recipients", json=payload, headers=auth_headers)
recipient_data = response.json()
if "message" in recipient_data:
  print(recipient_data["message"])
else:
  if len(recipient_data["recipients"]) > 0:
    print("Recipient links:\r\n-", '\r\n- '.join(map(lambda x: x["first_name"]+" "+x["last_name"]+": \t"+x["document_url"], recipient_data["recipients"])))
  else:
    raise Exception("Could not get recipients for document, please check the input data")